<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/PERCardio_Feb22_Drug_Data_1_0_(Write).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Read into one dataset and get its indexes

Note: get the exact file name to read

In [213]:
#import chardet
#with open('DOMComprasCardio.csv', 'rb') as rawdata:
#    result = chardet.detect(rawdata.read(100000))
#result
#PERantineoplastics1.xls

In [214]:
#!pip install --upgrade xlrd

In [215]:
import pandas as pd
#OrgData = pd.read_csv('DOMComprasCardio.csv',encoding='utf-8')
OrgData = pd.read_excel('PERCardio Feb22.xls')
OrgData.head(1)

,RazonSocial,Código Entidad,Entidad,Código Disa/Diresa,Establecimiento,Código Unidad Ejecutora,Punto de Entrega,Identificación Proceso,Código Producto,Producto,Cantidad,Precio Unitario Referencial,Monto Total Referencial,Precio Unitario Ofertado,Monto Total Ofertado,Pais
0,ALBIS S.A.,1,MINSA,2,DIRESA ANCASH,002,DIRECCIÓN REGIONAL DE SALUD ANCASH,LP SIP N 12 2012,4523,LOSARTAN 50 MG TAB,6000,0.0328,196.8,0.030572,183.431975,NaN


Note: get the special column name to get the dirty names

In [216]:
OrgDataDirtyName = OrgData['Producto']

In [217]:
len(OrgDataDirtyName) # length of drug names

3263

In [218]:
OrgDataDirtyName = list(set(OrgDataDirtyName))
len(OrgDataDirtyName) # length of drug names after remove duplicates

5

In [219]:
# uppercase everything
OrgDataDirtyNameU = []
for i in OrgDataDirtyName:
  OrgDataDirtyNameU.append(i.upper())

In [220]:
OrgDataDirtyNameU[:5]

['HIDROCLOROTIAZIDA 25 MG TAB',
 'HIDROCLOROTIAZIDA 12.5 MG TAB',
 'AMLODIPINO 10 MG TAB',
 'AMLODIPINO (COMO BESILATO) 5 MG TAB',
 'LOSARTAN 50 MG TAB']

translate from Spainish to English

In [221]:
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
example = OrgDataDirtyNameU[0]
translation = translator.translate(example, dest='en')
print(translation.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
HYDROCHLOROTHIAZIDE 25 MG TAB


In [222]:
OrgDataDirtyNameUEng = []
for item in OrgDataDirtyNameU:
  translation = translator.translate(item, dest='en')
  OrgDataDirtyNameUEng.append(translation.text)
# runs 1 mins

In [223]:
OrgDataDirtyNameUEng[:5]

['HYDROCHLOROTHIAZIDE 25 MG TAB',
 'HYDROCHLOROTHIAZIDE 12.5 MG TAB',
 'AMLODIPINE 10 MG TAB',
 'AMLODIPINE (COMO BESYLATO) 5 MG TAB',
 'LOSARTAN 50 MG TAB']

In [224]:
# get the orignila index about medicine name for each drug name 

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

OrgDataDirtyNameUESpl = []
for item in OrgDataDirtyNameUEng:
  tokenlist = tokenizer.tokenize(item)
  OrgDataDirtyNameUESpl.append(tokenlist)

In [225]:
OrgDataDirtyNameUESpl[:1]

[['HYDROCHLOROTHIAZIDE', '25', 'MG', 'TAB']]

In [226]:
OrgDataDirtyNameIndex  = []
for item in OrgDataDirtyNameUESpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  OrgDataDirtyNameIndex.append(templist)

In [227]:
OrgDataDirtyNameIndex[:5]

[['HYDROCHLOROTHIAZIDE'],
 ['HYDROCHLOROTHIAZIDE'],
 ['AMLODIPINE'],
 ['AMLODIPINE', 'BESYLATO'],
 ['LOSARTAN']]

In [228]:
# delete the information in each piece of drug name after SIMILAR
def DelSIMILAR(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'SIMILAR':
      finalindex = subindex
  return subitem[:finalindex]

In [229]:
# delete the information in each piece of drug name after EQUIVALENT
def DelEQUIVALENT(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'EQUIVALENT':
      finalindex = subindex
  return subitem[:finalindex]

In [230]:
OrgDataDirtyNameIndexDel = []
for item in OrgDataDirtyNameIndex:
  add = DelSIMILAR(item)
  add = DelEQUIVALENT(add)
  OrgDataDirtyNameIndexDel.append(add)

In [231]:
OrgDataDirtyNameIndexDel[:5]

[['HYDROCHLOROTHIAZIDE'],
 ['HYDROCHLOROTHIAZIDE'],
 ['AMLODIPINE'],
 ['AMLODIPINE', 'BESYLATO'],
 ['LOSARTAN']]

## Part 2: Set up indexes for clean catagol drug names

In [120]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [121]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [122]:
CatalogNameUS[:5] # after split by ","

[['LEVOTHYROXINE 0.375 MG', ' TABLET', ' TBD'],
 ['AMLODIPINE 2.5 MG + VALSARTAN 80 MG', ' TABLET', ' TBD'],
 ['LOSARTAN 100 MG', ' TABLET', ' TBD'],
 ['ATORVASTATIN 10 MG', ' TABLET', ' TBD'],
 ['FLUDROCORTISONE 0.1 MG', ' TABLET', ' 50 TABLETS BOTTLE']]

In [123]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])

In [124]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [125]:
PresentationForm[:5]# the presentation form

[' TABLET (CHEWABLE)',
 ' SOLUTION FOR INFUSION',
 ' INJECTION',
 ' TABLET (COATED)',
 ' SOLUTION FOR INJECTABLE. 20MG/10ML BOTTLE']

In [126]:
PresentationDiction = []

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

for item in PresentationForm:
  tokenlist = tokenizer.tokenize(item)
  PresentationDiction.append(tokenlist)

In [127]:
PresentationDiction2 = []

for item in PresentationDiction:
  for subitem in item:
    if len(subitem) > 3:
      PresentationDiction2.append(subitem)

In [128]:
PresentationDiction2 = list(set(PresentationDiction2))

In [129]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [130]:
print(len(MainMedicine), MainMedicine[:5])

462 ['LEVOTHYROXINE 0.375 MG', 'AMLODIPINE 2.5 MG + VALSARTAN 80 MG', 'LOSARTAN 100 MG', 'ATORVASTATIN 10 MG', 'FLUDROCORTISONE 0.1 MG']


In [131]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [132]:
MainMedicineSpl[:5]

[['LEVOTHYROXINE', '0', '.', '375', 'MG'],
 ['AMLODIPINE', '2', '.', '5', 'MG', '+', 'VALSARTAN', '80', 'MG'],
 ['LOSARTAN', '100', 'MG'],
 ['ATORVASTATIN', '10', 'MG'],
 ['FLUDROCORTISONE', '0', '.', '1', 'MG']]

In [133]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  MainMedicineIndex.append(templist)

In [134]:
MainMedicineIndex[:5]

[['LEVOTHYROXINE'],
 ['AMLODIPINE', 'VALSARTAN'],
 ['LOSARTAN'],
 ['ATORVASTATIN'],
 ['FLUDROCORTISONE']]

In [135]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [136]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [137]:
len(CatalogIndexData_df)

462

In [138]:
CatalogIndexData_df[:5]

,Catalog Name,Catalog Index
0,"LEVOTHYROXINE 0.375 MG, TABLET, TBD",[LEVOTHYROXINE]
1,"AMLODIPINE 2.5 MG + VALSARTAN 80 MG, TABLET, TBD","[AMLODIPINE, VALSARTAN]"
2,"LOSARTAN 100 MG, TABLET, TBD",[LOSARTAN]
3,"ATORVASTATIN 10 MG, TABLET, TBD",[ATORVASTATIN]
4,"FLUDROCORTISONE 0.1 MG, TABLET, 50 TABLETS BOTTLE",[FLUDROCORTISONE]


In [139]:
# create a list for main medicine from catalog name data set
CatalogMedName = []
for item in MainMedicineIndex:
  for subitem in item:
    CatalogMedName.append(subitem)

In [140]:
CatalogMedName = list(set(CatalogMedName))
len(CatalogMedName)

206

In [141]:
CatalogMedName[:3]

['DETEMIR', 'ASPARGINASE', 'PHOSPHATE']

## Part 3: Write the Index Data Set According to the Catalog Name Index

In [232]:
# for the vocabulaies which are included in the medicine index get from clean data set, keep them in the 
# Chile 2 data index
def KeepMetAnd(subitem):
  addlist = []
  for element in subitem:
    if element in CatalogMedName:
      addlist.append(element)
  return addlist

In [233]:
DatafinIndex = []
for sublist in OrgDataDirtyNameIndexDel:
  add = KeepMetAnd(sublist)
  DatafinIndex.append(add)

In [234]:
# count the number of Chile index sub list which is null []
count = 0
for i in DatafinIndex:
  if not i:
    count += 1

the length of total drug names and null index

In [235]:
print(len(DatafinIndex), count) 

5 0


In [236]:
nullindexlist = []
for i in range(0, len(DatafinIndex)):
  if not DatafinIndex[i]:
    nullindexlist.append(i)

In [52]:
!pip install -U spacy
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.4 MB/s 
     |████████████████████████████████| 6.0 MB 12.7 MB/s 
     |████████████████████████████████| 13.5 MB 31.0 MB/s 
     |████████████████████████████████| 71 kB 11.0 MB/s 
     |████████████████████████████████| 188 kB 59.0 MB/s 
     |████████████████████████████████| 660 kB 54.3 MB/s 
     |████████████████████████████████| 10.1 MB 36.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.1
    Uninstalling pydantic-1.9.1:
      Successfully uninstalled pydantic-1.9.1
  Attempting uninstall: thinc
    Found e

In [53]:
## Install scispaCy models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_craft_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_jnlpba_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.6 MB 1.8 MB/s 
     |████████████████████████████████| 5.8 MB 6.6 MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.4.0-py3-none-any.whl size=15660350 sha256=3ec2988cda71fdbfde1bae0bbc5e234264a5ebfac8beb2239de5ba1eefdccf56
  Stored in directory: /root/.cache/pip/wheels/31/4b/3a/03f2732eb69789a7df3bf29c1bc010f660ada5f3c295d4073c
Successfully built en-core-sci-sm
  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scisp

In [147]:
import scispacy
import spacy

#Core models
import en_core_sci_sm
import en_core_sci_lg

#NER specific models
import en_ner_craft_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_bionlp13cg_md

#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

In [148]:
#Load specific model and pass text through
nlp = en_core_sci_lg.load()

use spacy nlp to get informations from the drug names which with null index

In [237]:
DatadrugOutDataName = []
for nullindex in nullindexlist:
  tempdrugname = OrgDataDirtyNameIndexDel[nullindex]
  tempdrugname_str = str(' '.join(tempdrugname))
  doc = nlp(tempdrugname_str)
  temptext = doc.text
  DatadrugOutDataName.append(temptext)

In [238]:
drugOutDataNameNullNumber = 0

for i in DatadrugOutDataName:
  if not i:
    drugOutDataNameNullNumber += 1

the number of drug name which are still null

In [239]:
drugOutDataNameNullNumber

0

In [240]:
# for these null element in list, use Chile1drugOutDataName to complete them
j = 0
for i in nullindexlist:
  DatafinIndex[i] = DatadrugOutDataName[j]
  j += 1

In [241]:
for index in range(len(DatafinIndex)):
  if not DatafinIndex[index]:
    DatafinIndex[index] = OrgDataDirtyNameIndexDel[index]

In [242]:
nullindex = []
for index in range(len(DatafinIndex)):
  if not DatafinIndex[index]:
    nullindex.append(index)

In [243]:
nullindex

[]

In [244]:
PreList = []

for item in OrgDataDirtyNameUESpl:
  addlist = []
  for subitem in item:
    if subitem in PresentationDiction2:
      addlist.append(subitem)
  PreList.append(list(set(addlist)))

In [245]:
WeightList = []

for item in OrgDataDirtyNameUESpl:
  addlist = []
  for subitem in range(0, len(item)):
    if item[subitem].isdigit():
      if subitem + 1 < len(item):
        if item[subitem + 1].lower() in ['mg', 'ml']:
          addlist.append(item[subitem])
          addlist.append(item[subitem + 1])
  WeightList.append(list(set(addlist)))

In [246]:
print(len(DatafinIndex), len(PreList), len(WeightList))

5 5 5


In [247]:
# for catalog name index dataframe, transform list to string
str_DatafinIndex = []
for item in DatafinIndex:
  if not isinstance(item, str):
    addsub = []
    for subitem in item:
      if subitem:
        addsub.append(subitem)
        newitem = ' '.join(addsub)
  else:
    newitem = item

  str_DatafinIndex.append(newitem)

In [248]:
str_DatafinIndex[:4]

['HYDROCHLOROTHIAZIDE', 'HYDROCHLOROTHIAZIDE', 'AMLODIPINE', 'AMLODIPINE']

In [249]:
str_PreList = []

for item in PreList:
  newitem = ' '.join(item)
  str_PreList.append(newitem)

In [250]:
str_WeightList = []

for item in WeightList:
  newitem = ' '.join(item)
  str_WeightList.append(newitem)

In [251]:
DrugDataWrite = {"DOMComprasCardio Dirty Name English": OrgDataDirtyNameUEng, "Final Drug Index": str_DatafinIndex, "Presentation Form": str_PreList, "Weight": str_WeightList}


In [252]:
DrugDataWrite_df = pd.DataFrame(DrugDataWrite)

In [253]:
DrugDataWrite_df

,DOMComprasCardio Dirty Name English,Final Drug Index,Presentation Form,Weight
0,HYDROCHLOROTHIAZIDE 25 MG TAB,HYDROCHLOROTHIAZIDE,,25 MG
1,HYDROCHLOROTHIAZIDE 12.5 MG TAB,HYDROCHLOROTHIAZIDE,,MG 5
2,AMLODIPINE 10 MG TAB,AMLODIPINE,,10 MG
3,AMLODIPINE (COMO BESYLATO) 5 MG TAB,AMLODIPINE,,MG 5
4,LOSARTAN 50 MG TAB,LOSARTAN,,50 MG


In [254]:
DrugDataWrite_df.to_excel("PERCardio Feb22 (Write).xlsx", sheet_name = 'PERCardio Feb22')  